   1  : 1.58,
   0.1: 2.71,
  0.05: 3.84, 
  0.01: 6.63,
 0.005: 7.88, 
 0.001: 10.83

In [35]:
# 2.71 - 1.13

In [1]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [2]:
data = pd.read_excel("komentar/data_clean.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.tail()

,Unnamed: 0,nomer,label,komentar
4751,NaN,4751,spam,maa a da engenalan a e a jg ga percaya paydara...
4752,NaN,4752,spam,hay guys punya produk bagus banget buat tubuh ...
4753,NaN,4753,non spam,aliando jadi rem gitu iih
4754,NaN,4754,non spam,gak main film aja sih sama raffi jadi pasang k...
4755,NaN,4755,spam,kkk uu r tkk gt kkk uu ks tss sr gg rr kk rukk...


# Pemodelan

## TF-IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()#
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(4756, 14052)


## Pemodelan MNB 

In [7]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Pemodelan dan Validasi Model

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.001
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
# print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(alpha=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        #latih model
        clf.fit(X_train,y_train)
        
        #prediksi
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
#     print('alpha   :',alpha)
#     print('     akurasi :',rerata_akurasi)
#     print("="*100)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
#         print(alpha, ' | ', rerata_akurasi)
        print("")

        pass

ditt = {
    "alpha":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................


,alpha,akurasi
0,0.001,0.911907
1,0.002,0.914643
2,0.003,0.915484
3,0.004,0.915695
4,0.005,0.917587
5,0.006,0.918008
6,0.007,0.918009
7,0.008,0.918640
8,0.009,0.919060
9,0.010,0.920111


In [10]:
print(alpha_list[index])
print(akurasi[index])

0.296
0.9268460858027421


In [9]:
print(alpha_list[index])
print(akurasi[index])

0.296
0.9268460858027421


# Menyimpan Model

In [12]:
clf.set_params(alpha=alpha_list[index])
clf.fit(X, y)
dump(clf, "model/mnb.m")

['model/mnb.m']